In [1]:
import pandas as pd
import numpy as numpy
from importlib import reload
from tqdm import tqdm_notebook as tqdm
import time
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf

import pdaactconn as pc
from trialexplorer import AACTStudySet

In [2]:
conn = pc.AACTConnection(source=pc.AACTConnection.REMOTE)
ss = AACTStudySet.AACTStudySet(conn=conn, 
                               tqdm_handler=tqdm)
ss.add_constraint("study_type = 'Interventional'")
ss.add_constraint("results_first_submitted_date is not null")
ss.add_constraint("enrollment_type = 'Actual'")
ss.add_constraint("enrollment >= 10")
ss.add_constraint("enrollment <= 500")
ss.load_studies()

30462 studies loaded!


In [3]:
ss.add_dimensions(['baseline_measurements', 'result_groups', 'outcome_analyses'])
ss.refresh_dim_data()
rg = ss.dimensions['result_groups']
bm = ss.dimensions['baseline_measurements']

Successfuly added these 3 dimensions: ['baseline_measurements', 'result_groups', 'outcome_analyses']
Failed to add these 0 dimensions: []


Syncing the temp table temp_cur_studies in 61 chunks x 500 records each

Creating index on the temp table
 - Loading dimension baseline_measurements
 -- Loading raw data
 -- Sorting index
 - Loading dimension result_groups
 -- Loading raw data
 -- Sorting index
 - Loading dimension outcome_analyses
 -- Loading raw data
 -- Sorting index


In [4]:
#Get studies with exactly two treatment groups besides "Total" and grab necessary columns
combined_measures = pd.merge(rg.data, bm.data, left_on = ['nct_id', 'id'], right_on = ['nct_id', 'result_group_id'])
combined_measures = combined_measures[combined_measures.title_x != 'Total']
num_groups = combined_measures.groupby('nct_id').ctgov_group_code_x.nunique()
num_groups = pd.DataFrame(num_groups)
num_groups.columns.values[0] = 'n_groups'
combined_measures = combined_measures.merge(num_groups, on=['nct_id'])
combined_measures = combined_measures[combined_measures.n_groups==2]
study_balance_dat = combined_measures[['ctgov_group_code_x', 'classification', 'category', 'title_y',
                                       'param_type', 'param_value_num',
                                       'dispersion_type', 'dispersion_value_num']]

In [5]:
study_balance_dat.title_y.value_counts().head(10)
#We'll go with age and sex for now

Age                           55756
Sex: Female, Male             54034
Race (NIH/OMB)                41851
Region of Enrollment          24019
Race/Ethnicity, Customized    16978
Ethnicity (NIH/OMB)           12430
Age, Customized                6001
Gender                         1876
Sex/Gender, Customized          880
Weight                          879
Name: title_y, dtype: int64

In [6]:
sb_age = study_balance_dat[study_balance_dat.title_y == 'Age']
sb_age_cat = sb_age[sb_age.param_type=='Count of Participants']
sb_age_con = sb_age[sb_age.param_type=='Mean']
print(sb_age_cat.index.nunique())
print(sb_age_con.index.nunique())
#We'll choose the continuous version for now, since it has more

5218
10865


In [7]:
#Sex is always a count variable; only variability is capitalization of "category" field
#Get rid of rows that aren't sex or age
age_condition1 = study_balance_dat.title_y == 'Age'
age_condition2 = study_balance_dat.param_type == 'Mean'
sex_condition = study_balance_dat.title_y == 'Sex: Female, Male'
full_condition = (age_condition1 & age_condition2) | sex_condition
small_balance_dat = study_balance_dat[full_condition]
small_balance_dat.head(20)

,ctgov_group_code_x,classification,category,title_y,param_type,param_value_num,dispersion_type,dispersion_value_num
nct_id,,,,,,,,
NCT00000135,B2,,Male,"Sex: Female, Male",Count of Participants,94.0,,NaN
NCT00000135,B2,,Female,"Sex: Female, Male",Count of Participants,11.0,,NaN
NCT00000135,B1,,Male,"Sex: Female, Male",Count of Participants,93.0,,NaN
NCT00000135,B1,,Female,"Sex: Female, Male",Count of Participants,11.0,,NaN
NCT00000136,B2,,Male,"Sex: Female, Male",Count of Participants,116.0,,NaN
NCT00000136,B2,,Female,"Sex: Female, Male",Count of Participants,11.0,,NaN
NCT00000136,B1,,Male,"Sex: Female, Male",Count of Participants,98.0,,NaN
NCT00000136,B1,,Female,"Sex: Female, Male",Count of Participants,9.0,,NaN
NCT00000143,B2,,Male,"Sex: Female, Male",Count of Participants,25.0,,NaN


In [8]:
#get studies that have both measures we're using
num_measures = small_balance_dat.groupby('nct_id').title_y.nunique()
num_measures = pd.DataFrame(num_measures)
num_measures.columns.values[0] = 'n_measures'
small_balance_dat = small_balance_dat.merge(num_measures, on=['nct_id'])
small_balance_dat = small_balance_dat[small_balance_dat.n_measures==2]

#assert that each study has 6 rows now--2 sex and 1 age for each of the 2 arms
num_rows = small_balance_dat.groupby('nct_id').title_y.count()
num_rows = pd.DataFrame(num_rows)
num_rows.columns.values[0] = 'n_rows'
small_balance_dat = small_balance_dat.merge(num_rows, on=['nct_id'])
small_balance_dat = small_balance_dat[small_balance_dat.n_rows == 6]

In [9]:
#handle aforementioned capitalization issue
small_balance_dat.category = small_balance_dat.category.str.lower()

In [10]:
def calculate_imbalance(study_frame):
    nctid = study_frame.index[0]
    group_codes = study_frame.ctgov_group_code_x.unique()
    group1 = study_frame[study_frame.ctgov_group_code_x == group_codes[0]]
    group2 = study_frame[study_frame.ctgov_group_code_x == group_codes[1]]
    
    sex_imbalance = calculate_sex_imbalance(group1, group2)
    age_imbalance = calculate_age_imbalance(group1, group2)
    
    return([nctid, sex_imbalance, age_imbalance])

def calculate_sex_imbalance(arm1, arm2):
    arm1_sex = arm1[arm1.title_y == 'Sex: Female, Male']
    arm2_sex = arm2[arm2.title_y == 'Sex: Female, Male']
    
    arm1_size = arm1_sex.param_value_num.sum()
    arm2_size = arm2_sex.param_value_num.sum()
    
    arm1_men = arm1_sex[arm1_sex.category == 'male'].param_value_num[0]
    arm2_men = arm2_sex[arm2_sex.category == 'male'].param_value_num[0]
    
    arm1_pct_men = arm1_men / arm1_size
    arm2_pct_men = arm2_men / arm2_size
    
    sex_imbalance = abs(arm1_pct_men - arm2_pct_men)
    return(sex_imbalance)

def calculate_age_imbalance(arm1, arm2):
    arm1_mean_age = arm1[arm1.title_y == 'Age'].param_value_num[0]
    arm2_mean_age = arm2[arm2.title_y == 'Age'].param_value_num[0]
    
    age_imbalance = abs(arm1_mean_age - arm2_mean_age) / (arm1_mean_age + arm2_mean_age)
    return(age_imbalance)

In [11]:
#Time to calculate imbalances
imbalance_dat = []
study_ids = small_balance_dat.index.unique()
n_studies = study_ids.shape[0]
print('Calculating imbalance for ' + str(n_studies) + ' studies')
for i in range(n_studies):
    current_id = study_ids[i]
    current_study = small_balance_dat[small_balance_dat.index == current_id]
    current_imbalances = calculate_imbalance(current_study)
    imbalance_dat.append(current_imbalances)
    if (i + 1) % 1000 == 0:
        print('Finished with ' + str(i + 1) + ' studies.')
imbalance_frame = pd.DataFrame(imbalance_dat, columns = ['nct_id', 'sex_imbalance', 'age_imbalance'])
imbalance_frame.head()

Calculating imbalance for 10322 studies
Finished with 1000 studies.
Finished with 2000 studies.
Finished with 3000 studies.
Finished with 4000 studies.
Finished with 5000 studies.
Finished with 6000 studies.
Finished with 7000 studies.
Finished with 8000 studies.
Finished with 9000 studies.
Finished with 10000 studies.


,nct_id,sex_imbalance,age_imbalance
0,NCT00000371,0.174603,0.011841
1,NCT00001586,0.012755,0.019368
2,NCT00001596,0.152174,0.053174
3,NCT00001656,0.128205,0.044898
4,NCT00001723,0.010000,0.004457


In [12]:
imbalance_frame.describe()

,sex_imbalance,age_imbalance
count,10319.000000,10314.000000
mean,0.078111,0.027772
std,0.099090,0.050822
min,0.000000,0.000000
25%,0.010971,0.006076
50%,0.047619,0.014624
75%,0.104933,0.030487
max,1.000000,0.975217


In [13]:
imbalance_frame.corr()
#I think covariance would be 0 if randomization were sole cause of imbalance

,sex_imbalance,age_imbalance
sex_imbalance,1.000000,0.174346
age_imbalance,0.174346,1.000000


In [14]:
oa = ss.dimensions['outcome_analyses']
oa.data.reset_index(inplace=True)
#Note: We talked about using t-vals, but I didn't see t-vals in outcome analyses table. 
#p vals are monotone in abs(t val) so I think this is a decent proxy for now
#Not great to run a linear regression on a bounded target though
p_val_dat = oa.data.drop_duplicates(subset = 'nct_id', keep = 'first')[['nct_id', 'p_value']]
p_val_dat.head()

,nct_id,p_value
0,NCT00000378,0.050
1,NCT00000392,0.180
2,NCT00001656,0.730
13,NCT00001723,0.007
14,NCT00001959,0.160


In [15]:
print(imbalance_frame.shape[0])
print(p_val_dat.shape[0])

10322
10705


In [16]:
regression_dat = imbalance_frame.merge(p_val_dat, how = 'inner', on = ['nct_id'])
regression_dat.shape[0]
#THIS IS NOT GREAT!

4563

In [17]:
regression_dat.head()

,nct_id,sex_imbalance,age_imbalance,p_value
0,NCT00001656,0.128205,0.044898,0.730
1,NCT00001723,0.010000,0.004457,0.007
2,NCT00003222,0.217033,0.017857,0.004
3,NCT00004980,0.014493,0.005666,0.010
4,NCT00005947,0.000000,0.006983,0.010


In [18]:
print(regression_dat.sex_imbalance.isna().sum())
print(regression_dat.age_imbalance.isna().sum())
print(regression_dat.p_value.isna().sum())
#Still a lot of missing p values

1
3
572


In [19]:
regression_dat.dropna(inplace=True)
regression_dat.shape

(3988, 4)

In [20]:
#On this first pass at least, Rong's hypothesis about the big std.err is true
linmod = smf.ols('p_value ~ sex_imbalance + age_imbalance', data=regression_dat).fit()
print(linmod.summary())

                            OLS Regression Results                            
Dep. Variable:                p_value   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.6362
Date:                Fri, 15 Nov 2019   Prob (F-statistic):              0.529
Time:                        21:07:46   Log-Likelihood:                -1047.5
No. Observations:                3988   AIC:                             2101.
Df Residuals:                    3985   BIC:                             2120.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.2603      0.007     37.582

In [21]:
#Try fitting 4 more models: both univariates, log(p), and only p in [0.01, 0.1]
#This block does univariates
linmod_just_sex = smf.ols('p_value ~ sex_imbalance', data=regression_dat).fit()
linmod_just_age = smf.ols('p_value ~ age_imbalance', data=regression_dat).fit()
print(linmod_just_sex.summary())
print(linmod_just_age.summary())

                            OLS Regression Results                            
Dep. Variable:                p_value   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                 0.0001307
Date:                Fri, 15 Nov 2019   Prob (F-statistic):              0.991
Time:                        21:07:46   Log-Likelihood:                -1048.1
No. Observations:                3988   AIC:                             2100.
Df Residuals:                    3986   BIC:                             2113.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.2633      0.006     41.184

In [22]:
#Make log(p_val) the target
#Some p vals in data are 0, so add tiny bit to avoid error
regression_dat['log_p_value'] = numpy.log(regression_dat.p_value + 0.0001)
linmod_logp = smf.ols('log_p_value ~ sex_imbalance + age_imbalance', data=regression_dat).fit()
print(linmod_logp.summary())

                            OLS Regression Results                            
Dep. Variable:            log_p_value   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.586
Date:                Fri, 15 Nov 2019   Prob (F-statistic):             0.0278
Time:                        21:07:46   Log-Likelihood:                -9623.8
No. Observations:                3988   AIC:                         1.925e+04
Df Residuals:                    3985   BIC:                         1.927e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -3.1539      0.059    -53.020

In [23]:
p_condition = (regression_dat.p_value < .1) & (regression_dat.p_value > .01)
p_condition.value_counts()

False    3000
True      988
Name: p_value, dtype: int64

In [24]:
small_regression_dat = regression_dat[p_condition]
linmod_limited_p = smf.ols('p_value ~ sex_imbalance + age_imbalance', data=small_regression_dat).fit()
print(linmod_limited_p.summary())

                            OLS Regression Results                            
Dep. Variable:                p_value   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.992
Date:                Fri, 15 Nov 2019   Prob (F-statistic):              0.137
Time:                        21:07:46   Log-Likelihood:                 2494.0
No. Observations:                 988   AIC:                            -4982.
Df Residuals:                     985   BIC:                            -4967.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0460      0.001     54.084